In [29]:
# Cell 1: Install dependencies
!pip install pathway bokeh panel --quiet

# Cell 2: Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import pathway as pw
import bokeh.plotting as bk
import bokeh.io as bio
from bokeh.layouts import column
from bokeh.models import ColumnDataSource
import panel as pn
pn.extension()

# Cell 3: Load dataset
df = pd.read_csv("/content/dataset.csv")

# Preprocess datetime
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], dayfirst=True, errors='coerce')

df.sort_values('Timestamp', inplace=True)
df.reset_index(drop=True, inplace=True)

# Encode categorical variables
df['VehicleTypeWeight'] = df['VehicleType'].map({'bike': 0.5, 'car': 1.0, 'truck': 1.5})
df['TrafficLevel'] = df['TrafficConditionNearby'].map({'low': 0.5, 'medium': 1.0, 'high': 1.5})

# Set base price
BASE_PRICE = 10

# Cell 4: Define pricing models
def linear_price_model(row, prev_price):
    alpha = 2
    occupancy_ratio = row['Occupancy'] / row['Capacity']
    return prev_price + alpha * occupancy_ratio

def demand_based_price(row):
    alpha, beta, gamma, delta, epsilon = 1, 1.5, 0.8, 2, 1.2
    demand = (
        alpha * (row['Occupancy'] / row['Capacity']) +
        beta * row['QueueLength'] -
        gamma * row['TrafficLevel'] +
        delta * row['IsSpecialDay'] +
        epsilon * row['VehicleTypeWeight']
    )
    norm_demand = np.tanh(demand / 10)  # keeps it bounded between -1 and 1
    price = BASE_PRICE * (1 + 0.5 * norm_demand)
    return round(min(max(price, 5), 20), 2)  # bound price between $5 and $20

# Cell 5: Initialize simulation state
price_state = {lot: BASE_PRICE for lot in df['SystemCodeNumber'].unique()}
source = ColumnDataSource(data={'time': [], 'price': []})
bk.output_notebook()

# Cell 6: Stream and plot in real time using Panel & Bokeh
def simulate_streaming(df):
    plot = bk.figure(x_axis_type='datetime', title="Real-Time Parking Price (Lot 1)",
                     height=300, width=800)
    plot.line(source=source, x='time', y='price', line_width=2, color="navy")
    panel = pn.panel(plot)

    for i, row in df.iterrows():
        lot_id = row['SystemCodeNumber']
        prev_price = price_state[lot_id]

        # Choose model here: linear or demand-based
        # new_price = linear_price_model(row, prev_price)
        new_price = demand_based_price(row)

        price_state[lot_id] = new_price

        if lot_id == list(price_state.keys())[0]:  # only plot first lot for now
            source.stream({'time': [row['Timestamp']], 'price': [new_price]}, rollover=300)

    return panel

simulate_streaming(df.head(100))


Bokeh(figure)

Installing Pathway, Bokeh, and Panel...
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.31.0, but you have google-cloud-bigquery 3.29.0 which is incompatible.
Installation complete.

--- Data Upload Instruction ---
Please upload 'dataset.csv' to your Google Colab session storage.
You can do this by clicking the 'Files' icon on the left sidebar (folder icon),
then clicking the 'Upload to session storage' icon (page with an arrow pointing up).
Make sure the file is named 'dataset.csv'.

Ingesting data with Pathway from 'dataset.csv'...
Error loading dataset.csv. Make sure it's uploaded correctly. Error: module 'pathway' has no attribute 'read'
Data ingestion setup complete. Applying feature engineering...


AttributeError: 'NoneType' object has no attribute 'map'